# Creating a Tidy Dataset

In this notebook we will load in the POLA dataset and transform it into a "tidy" dataset. A common standard for data analysis is for each row to be an observation and each coumn to represent a measured value. Recently, this form has come to be known as the "tidy" form, after a 2013 article describing the format. 

The POLA data, in contrast, is in "long form", where one column describes the type of measurement, and another column has the result value for multiple measurements. Furthermore, the POLA data is split across multipe files. The whole dataset is small, less than 1m rows, so we will combine all of the files into a single dataframe, then convert to tidy format. 


In [1]:
# Imports should be collected at the top, as a standard programming convention

# This turns off a warning about a deprecation in Pandas, from coude referenced by Statsmodel. 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# These imports are the most common for data analysis, 
# and they are given standard aliases. 
import pandas as pd
import numpy as np
import seaborn as sns

# For other imports, we only need one or two 
# functions, rather than the whole module. 
from scipy.stats import zscore
from pathlib import Path
from os import environ

First we need to figure out where the data is. It can be in difference locations, depending on wether we are running on Google Colab, or locally with Jupyter. 

In [2]:
# Mount Google Drive into our VM, so we can access stored files. 
# The try/catch allows this code to detect if it is not running 
try:
    from google.colab import drive
    drive.mount('/gdrive')
except ModuleNotFoundError:
    pass # We aren't running in GOogle colab environment

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
# Load the datasets, combine them, and do some cleanup. 

if Path('/gdrive').is_dir():
    # This branch handles the case that the notebook is running on Google Colab. 
    data_dir = Path('/gdrive/Shared drives/Wood/Data/')
    
elif environ.get('WOODPLC_DATA_DIR'):
    # This branch handles the case of the notebook running locally. 
    # To run locally, download the Woodplc data directory, or setup Google Drive Stream to mount it, 
    # then set the environmental variable to the directory. For instance: 
    #     export WOODPLC_DATA_DIR='/Volumes/GoogleDrive/Shared drives/Wood/Data'
    data_dir = Path(environ.get('WOODPLC_DATA_DIR'))
    
pola_data_dir = data_dir.joinpath('POLA Water Quality Database Files/CTD Files Organized for Python/Ready for Access')



Now that we've figured where the data should be, we can load it. The POLA data source is in multiple files, one per year, but they are relatively small, so we will load them into a  single dataframe. After loading, we'll convert some date/times and force other clolumns to be numbers. 

In [4]:
# Load in each of the dataframe, into an arrah
frames = []
for f in Path(pola_data_dir).glob('*.csv'):
    frames.append(pd.read_csv(f, low_memory=False))

# BTW, The more pythonic way to create the frames list is with a list comprehension:
# frames = [pd.read_csv(f, low_memory=False) for f in Path(pola_data_dir).glob('*.csv')]

# Combine the individual data frames into a single dataframe. 
df = pd.concat(frames)

# combine the date and time into a string that can be parsed by pd.to_datetime
# We'll also have to get rid of the 'NR' string in some of the time value, since
# that will creash the datetime parser. 
datetime = df['Sample Date']+' ' + df['Sample Time'].replace({'NR':''})

df['datetime'] = pd.to_datetime( datetime )

# Convert fields that were read as strings into numbers.
df['Result'] = pd.to_numeric(df.Result, errors='coerce')
df['Depth'] = pd.to_numeric(df.Depth, errors='coerce')

df.to_csv(data_dir.joinpath('pola.csv')) #Save it for later. 
df.head()

,SampleID,Site Name,Sample Date,Sample Time,Depth,Depth Units,Depth_Category,Parameter,Result,Qualifier,Result Unit,Method,MDL,RL,Comments,datetime
0,LA01_20090121_Dissolved Oxygen_3.0m,LA01,2009-01-21,10:21,3.0,m,surface,Dissolved Oxygen,6.89082,NaN,mg/l,CTD,NaN,NaN,NaN,2009-01-21 10:21:00
1,LA01_20090121_Dissolved Oxygen_3.5m,LA01,2009-01-21,10:21,3.5,m,NaN,Dissolved Oxygen,7.47152,NaN,mg/l,CTD,NaN,NaN,NaN,2009-01-21 10:21:00
2,LA01_20090121_Dissolved Oxygen_4.0m,LA01,2009-01-21,10:21,4.0,m,NaN,Dissolved Oxygen,7.86144,NaN,mg/l,CTD,NaN,NaN,NaN,2009-01-21 10:21:00
3,LA01_20090121_Dissolved Oxygen_4.5m,LA01,2009-01-21,10:21,4.5,m,NaN,Dissolved Oxygen,7.51499,NaN,mg/l,CTD,NaN,NaN,NaN,2009-01-21 10:21:00
4,LA01_20090121_Dissolved Oxygen_5.0m,LA01,2009-01-21,10:21,5.0,m,NaN,Dissolved Oxygen,7.59701,NaN,mg/l,CTD,NaN,NaN,NaN,2009-01-21 10:21:00


After we have a clean dataset in the df variable, we will create alternative views, called t, for individual analyses. Note that the value `t` should not be carried from cell to cell. If you want to keep if for later cells, assign it to another name, so we'll save the final dataset to the name 'tidy'

In [5]:
# Here, we will create a "tidy" dataset, where each variable has its own column, 
# rather than having a single Results column for all of the different measurements

# Get a subset of columns. Copy, b/c we'll make changes. 
t = df[['Site Name','datetime','Parameter','Result', 'Depth']].copy()

# Change the column names, by getting rid of spaces and lowercasing. 
t.columns = [c.replace(' ','').lower() for c in t.columns]

# Build a replacement map to clean up the Parameter values. The
# original values have spaces in them, which don't make good names for columns. 
replace_map = { k:k.replace(' ','_').lower() for k in df.Parameter.unique() }

# Replace the values. 
t['parameter'] = t.parameter.replace(replace_map)

# We'll save this so we can examine it more later. 
t_i = t.copy()

t = ( t.groupby(['sitename','datetime','depth','parameter']) # Group rows by these four values
    .mean() # Compute the mean value of each group, for all other columns
    .unstack() )# pivot the last index column, parameter,  to column heading. 

# The resulting column headers has two levels, drop one of them, 
# 'result', which has only one value. 
tidy = t.droplevel(0, axis=1).reset_index()
tidy.reset_index().to_csv(data_dir.joinpath('pola_tidy.csv'),index=False)
tidy.head()


parameter,sitename,datetime,depth,dissolved_oxygen,fluorescence,ph,salinity,temperature,transmissivity,turbidity
0,AS1,2018-07-17 11:49:00,0.5,7.16056,2.9065,8.094,33.6155,19.9630,75.6053,0.671335
1,AS1,2018-07-17 11:49:00,1.0,7.17666,3.0533,8.090,33.6125,19.9335,75.3734,0.883069
2,AS1,2018-07-17 11:49:00,1.5,7.13837,3.4530,8.091,33.5674,19.7881,75.6926,0.883727
3,AS1,2018-07-17 11:49:00,2.0,7.10275,4.0010,8.088,33.5984,19.6458,75.8661,0.734444
4,AS1,2018-07-17 11:49:00,2.5,7.06786,4.6305,8.090,33.6125,19.5989,75.8419,0.817684


## Removing outliers

Unless you specifically want to study outliers, we can assume that measurements that are very distant from the mean represent measurement errors. If they are very far from the mean, they can overwhelm other analyses, so they should be removed. The methods we will use for removing outliers are eliminating impossible measurements, such as negative values for measurements that can't be negative, and removing measurements that are far from the mean.For the second method, we will convert the values to z-scores, which convert values into units of standard deviations, and then remove every measurement that is more the 6 standard deviations from the mean, which is z-scores outside of the range -6 to 6, a range that will include 99.9999998% of measurements if the measurements are Normally distributed. 


In [6]:
tidy.describe()

parameter,depth,dissolved_oxygen,fluorescence,ph,salinity,temperature,transmissivity,turbidity
count,86883.000000,86191.000000,86191.000000,86181.000000,86184.000000,86191.000000,86191.000000,81818.000000
mean,7.210621,4.848246,1.800437,7.980318,33.296879,16.496583,70.444513,2.585758
std,22.492295,130.009563,2.164946,0.372699,8.930027,2.196529,11.199334,30.525673
min,0.500000,-38160.000000,-11.698200,-3.627000,2.060200,-98.976200,-61.806200,-258.485180
25%,3.022000,3.975900,0.791300,7.790000,33.180975,14.991900,65.297100,0.850299
50%,6.500000,5.465880,1.250900,7.950000,33.325000,16.163500,71.929000,1.219774
75%,10.500000,6.454395,2.147900,8.129000,33.441800,18.107000,77.611550,1.786680
max,3635.000000,24.022910,190.597600,9.112000,2000.000000,25.735000,112.832000,1485.114100


In [7]:
t = tidy.set_index(['sitename', 'datetime']).copy() # so the zscore operation wont try to process strings. 

# Disolved oxygen can't be less than zero, so replace these values with nan
t['dissolved_oxygen'] = t.dissolved_oxygen.mask(t.dissolved_oxygen < 0 )

# The outlier remove prcedure involves
# * compute the zscore of each column
# * convert the zscores to absolute values ( folding negative zscores to positives ) 
# * convert any remaining Nan ( not a number ) values to 6
# * Create an array of booleans indicating if the |zscore| > 6

is_outlier = np.nan_to_num(np.abs(zscore(t,nan_policy='omit')), nan=6) >= 6

# Replace values where is_outlier == True with nan
tidy = t.mask(is_outlier)

print("# Outliers", is_outlier.sum())
print(f"Original number of nans={sum(np.isnan(t.values.flatten()) )}, after outlier removal={sum(np.isnan(tidy.values.flatten()))}")

# Save the file
tidy.reset_index().to_csv(data_dir.joinpath('pola_tidy_no.csv'),index=False)

tidy.describe()

# Outliers 10033
Original number of nans=9532, after outlier removal=10033


parameter,depth,dissolved_oxygen,fluorescence,ph,salinity,temperature,transmissivity,turbidity
count,86879.000000,85891.000000,85935.000000,86078.000000,86179.000000,86189.000000,86157.000000,81723.000000
mean,7.062379,5.311845,1.738167,7.986383,33.245095,16.498197,70.473253,1.587678
std,4.619183,1.580799,1.544605,0.312214,0.613241,2.159556,11.105708,3.112880
min,0.500000,1.067220,-5.913700,6.081000,2.060200,8.846500,3.659200,-61.955521
25%,3.021500,3.987150,0.790200,7.791000,33.180900,14.991900,65.306100,0.849862
50%,6.500000,5.475370,1.248100,7.950000,33.325000,16.163500,71.934500,1.218972
75%,10.500000,6.458165,2.139000,8.130000,33.441800,18.107000,77.613600,1.782707
max,26.500000,12.799640,14.771200,9.112000,48.576000,25.735000,112.832000,182.780770


In [8]:
# Here are the measurements that were excluded for being outliers. 
outliers = t.where(is_outlier).stack().reset_index()[['parameter',0]]
outliers.columns = ['parameter','value']
print(len(outliers))
outliers.parameter.value_counts()

501


fluorescence        256
ph                  103
turbidity            95
transmissivity       34
salinity              5
depth                 4
dissolved_oxygen      2
temperature           2
Name: parameter, dtype: int64

Let's look at the process in more detail. Let's replay the first part of that process, to just after processing the column and parameter names, but before grouping and unstacking. 

In [9]:
t = df[['Site Name','datetime','Parameter','Result', 'Depth']].copy()

# Change the column names, by getting rid of spaces and lowercasing. 
t.columns = [c.replace(' ','').lower() for c in t.columns]

# Build a replacement map to clean up the Parameter values. The
# original values have spaces in them, which don't make good names for columns. 
replace_map = { k:k.replace(' ','_').lower() for k in df.Parameter.unique() }

# Replace the values. 
t['parameter'] = t.parameter.replace(replace_map)

ti = t # Save the name

ti.head()


,sitename,datetime,parameter,result,depth
0,LA01,2009-01-21 10:21:00,dissolved_oxygen,6.89082,3.0
1,LA01,2009-01-21 10:21:00,dissolved_oxygen,7.47152,3.5
2,LA01,2009-01-21 10:21:00,dissolved_oxygen,7.86144,4.0
3,LA01,2009-01-21 10:21:00,dissolved_oxygen,7.51499,4.5
4,LA01,2009-01-21 10:21:00,dissolved_oxygen,7.59701,5.0


We've taken out all of the columns that we don't really need for analysis, and changed the parameter names. The ``parameter`` column has multiple values, describing the type of measurement in each row. The next step is to create groups for the main independent variables, which will form the index of the groups. These independent variables are the sitename, measurement time, depth, and the parameter name. 

The ``.groupby`` method creates groups of rows, and the ``.size()`` method of the GroupBy object gives us how many rows are in the group. Idealy, there should be one for each of the index values, but we should check, because having multiple values per index can be a problem. 


In [10]:
g = ti.groupby(['sitename','datetime','depth','parameter']) # Group rows by these four values
g.size().sort_values(ascending=False) # Some of the groups have size 2

sitename  datetime             depth  parameter       
LA05      2017-09-06 09:25:00  4.029  ph                  2
LA18      2017-10-05 09:52:00  5.500  temperature         2
                               6.000  dissolved_oxygen    2
                                      fluorescence        2
                                      ph                  2
                                                         ..
LA46      2009-03-18 09:42:00  9.000  temperature         1
                                      salinity            1
                                      ph                  1
                                      fluorescence        1
AS1       2018-07-17 11:49:00  0.500  dissolved_oxygen    1
Length: 598957, dtype: int64

Unfortunately, there are some groups with two records, so we will have to determine how to deal with these. We can get these records by their index and see if they actually have different values for the measurements. 

In [11]:
t = g.size().sort_values(ascending=False) # Re-group and save it. 
tg = ti.set_index(['sitename','datetime','depth','parameter']) # Set the index to the same as the group
idx = t[t>1].index # Get the index values for all groups with more than one record
tg.loc[idx].head(20) # Get the duplicated rows. 

result
sitename datetime            depth parameter                
LA05     2017-09-06 09:25:00 4.029 ph                 8.0530
                                   ph                 8.0700
LA18     2017-10-05 09:52:00 5.500 temperature       19.7766
                                   temperature       19.4142
                             6.000 dissolved_oxygen   6.1507
                                   dissolved_oxygen   7.6587
                                   fluorescence       0.4944
                                   fluorescence       4.0139
                                   ph                 8.1470
                                   ph                 8.1150
                                   salinity          33.3756
                                   salinity          33.2726
                                   temperature       19.8124
                                   temperature       19.3546
                                   transmissivity    68.8012
                                   transmissivity    73.8898
                             6.500 dissolved_oxygen   9.2481
                                   dissolved_oxygen   7.6143
                                   fluorescence       4.3693
                                   fluorescence       3.3531

The easiest way to deal with the duplicate readings is to average them. 

In [12]:
t = g.mean()
t

result
sitename datetime            depth parameter                  
AS1      2018-07-17 11:49:00 0.5   dissolved_oxygen   7.160560
                                   fluorescence       2.906500
                                   ph                 8.094000
                                   salinity          33.615500
                                   temperature       19.963000
...                                                        ...
LB23     2018-12-11 08:57:00 13.5  ph                 7.992000
                                   salinity          33.003500
                                   temperature       15.755400
                                   transmissivity    71.366000
                                   turbidity          1.870506

[598957 rows x 1 columns]

Now that we have only one record per index value, we can "pivot" the parameters column into column headings. The ``.unstack()`` method will take the last column in the index and rotate it so that the values of the column become column headings, and the value ( ``results`` ) for that row becomes the value under the new column created for the parameter. 

In [13]:
t = t.unstack()
t

result  ...          
parameter                          dissolved_oxygen  ... turbidity
sitename datetime            depth                   ...          
AS1      2018-07-17 11:49:00 0.5            7.16056  ...  0.671335
                             1.0            7.17666  ...  0.883069
                             1.5            7.13837  ...  0.883727
                             2.0            7.10275  ...  0.734444
                             2.5            7.06786  ...  0.817684
...                                             ...  ...       ...
LB23     2018-12-11 08:57:00 11.5           5.95369  ...  1.624902
                             12.0           6.02815  ...  1.814890
                             12.5           6.03424  ...  1.793472
                             13.0           6.02223  ...  1.850009
                             13.5           6.01688  ...  1.870506

[86883 rows x 7 columns]

That last table looks like what we want! But, there is one final thing to fix. Notice that the old column name for ``result`` is still there, just above "turbidity". Because the data frame may have had more than one column before we did the unstack, the ``.unstack()`` method creates a multi-level column index, which we can see with:

In [14]:
t.columns

MultiIndex([('result', 'dissolved_oxygen'),
            ('result',     'fluorescence'),
            ('result',               'ph'),
            ('result',         'salinity'),
            ('result',      'temperature'),
            ('result',   'transmissivity'),
            ('result',        'turbidity')],
           names=[None, 'parameter'])

If th table had originally had two columns, one for ``result`` and one for ``measurement_error``, the table would have to retail both of those values for each parameter after the ``.unstack()`` operation. But, we only have one, so we can get rid of the multi-level index with:

In [15]:
t = t.droplevel(0, axis=1)
t

parameter                           dissolved_oxygen  ...  turbidity
sitename datetime            depth                    ...           
AS1      2018-07-17 11:49:00 0.5             7.16056  ...   0.671335
                             1.0             7.17666  ...   0.883069
                             1.5             7.13837  ...   0.883727
                             2.0             7.10275  ...   0.734444
                             2.5             7.06786  ...   0.817684
...                                              ...  ...        ...
LB23     2018-12-11 08:57:00 11.5            5.95369  ...   1.624902
                             12.0            6.02815  ...   1.814890
                             12.5            6.03424  ...   1.793472
                             13.0            6.02223  ...   1.850009
                             13.5            6.01688  ...   1.870506

[86883 rows x 7 columns]

And now we have just one level for the columns:

In [16]:
t.columns

Index(['dissolved_oxygen', 'fluorescence', 'ph', 'salinity', 'temperature',
       'transmissivity', 'turbidity'],
      dtype='object', name='parameter')

Now our tidy version is ready for use. 